In [1]:
from frontend.database import session
from scripts import generate_fc_layer_circuit, benchmark_utils
from scripts.circuit import circuit
from scripts.particles import variables, enc_vec, enc_mat
from scripts.reusable_modules import oneb_adder, nb_adder
import numpy as np

In [2]:
from scripts.functions import reduce_add
inp = enc_vec(name='in', nb=4)
out = enc_vec(name='out', nb=3)
ra = reduce_add('adder', inp, out)
ra_obj = circuit('reduce_add', ra)
ra_obj.write_file(filename='./test.sheep')

# NB bit adder

In [3]:
import math
nb = 25
depth = int(math.floor(math.log(nb, 2)))
samples=2
inp = enc_vec(name='in', nb=nb)
out = enc_vec(name='out', nb=depth+1)
ra = reduce_add('adder', inp, out)
ra_obj = circuit('reduce_add', ra)
ra_obj.write_file(filename='./test.sheep')
processing_time = np.zeros(samples)
for idx in range(0, samples):
    inp_arr_val = list(map(lambda x : int(x), np.random.uniform(size=nb)> 0.5))
    inp_dict = inp.get_input_dict(inp_arr_val)
    inputs_file = benchmark_utils.write_inputs_file(inp_dict)
    results = benchmark_utils.run_circuit('./test.sheep',inputs_file,"bool","TFHE", eval_strategy='parallel')
    processing_time[idx] = results['Processing times (s)']['circuit_evaluation']
    out_vars = out.get_variables()
    err_str = str(''.join(results['Outputs'][var] for var in out_vars)[::-1]) +" -- "+ str(np.asarray(inp_arr_val).sum())
    assert int(''.join(results['Outputs'][var] for var in out_vars)[::-1], 2) == np.asarray(inp_arr_val).sum(), err_str
print(processing_time.mean())

1.0


# Encrypted Prediction as a Service

In [32]:
from scripts.nn_layer import linear_layer_1d
from tqdm import tqdm
nb = 10
samples = 1
inputs = enc_vec(name='nn_input', nb=nb)
depth = int(math.ceil(math.log(nb*1.0, 2)))
outputs = enc_vec(name='nn_outputs', nb=1)
outputs_sum = enc_vec(name='sum_outputs', nb=depth+1)
weight = np.asarray(list(map(lambda x : int(x), np.random.uniform(size=nb)> 0.5)))
layer = linear_layer_1d(name='linear', weight=weight,
                    inputs=inputs, outputs=outputs)
layer_obj = circuit('linear_layer', layer, const_inputs=[])
layer_obj.write_file(filename='./test_linear.sheep')
processing_time = np.zeros(samples)
for idx in tqdm(range(0, samples)):
    inp_arr_val = list(map(lambda x : int(x), np.random.uniform(size=nb)> 0.5))
    inp_dict = inputs.get_input_dict(inp_arr_val)
    inputs_file = benchmark_utils.write_inputs_file(inp_dict)
    results = benchmark_utils.run_circuit('./test_linear.sheep',inputs_file,"bool","TFHE",eval_strategy='parallel')
    processing_time[idx] = results['Processing times (s)']['circuit_evaluation']
    out_vars = outputs.get_variables()
    xor_val = (weight == np.asarray([x for x in inp_arr_val]))
    err_str = str(''.join(results['Outputs'][var] for var in out_vars)[::-1]) +" -- "+ str(np.asarray(xor_val).sum())
    assert int(''.join(results['Outputs'][var] for var in out_vars)[::-1], 2) == int(np.asarray(xor_val).sum()>nb/2), err_str
print(processing_time.mean())

100%|██████████| 1/1 [00:00<00:00,  1.06it/s]

0.4946


# Comparing Numbers (Ciphertext with Plaintext)

In [5]:
from scripts.reusable_modules import compare_cp
cp_inp_val = [1, 0, 1, 1, 1, 1, 1]#[x for x in bin(np.asarray(inp_arr_val).sum())][::-1][:-2] #map(lambda x: int(x), inp_arr_val)[::-1]
tgt = 126#len(cp_inp_val)/2
comp_inp = enc_vec(name='cp_inp',nb = len(cp_inp_val))
out_vec = enc_vec(name='cp_out', nb=1)
cp_circ = compare_cp(name='cp', inputs=(comp_inp, tgt), outputs=out_vec)
sum_obj = circuit('cp', cp_circ)
sum_obj.write_file(filename='./test_cp.sheep')
inputs_file = benchmark_utils.write_inputs_file(comp_inp.get_input_dict(cp_inp_val))
results = benchmark_utils.run_circuit('./test_cp.sheep',inputs_file,"bool","TFHE",eval_strategy='parallel')
processing_time[idx] = results['Processing times (s)']['circuit_evaluation']
print(int(results['Outputs'][out_vec.get_variables()[0]]) == int(int(''.join(map(lambda x : str(x), cp_inp_val)),2) >= tgt))
if int(results['Outputs'][out_vec.get_variables()[0]]) == 1:
    print str( int(int(''.join(map(lambda x : str(x), cp_inp_val[::-1])),2)))+"(CP) is greater than "+str(tgt)+str("(PT)")
else:
    print str( int(int(''.join(map(lambda x : str(x), cp_inp_val[::-1])),2)))+"(CP) is smaller than "+str(tgt)+str("(PT)")
print(results['Processing times (s)']['circuit_evaluation'])


True
125(CP) is smaller than 126(PT)
0.36815


In [31]:
from scripts.nn_layer import linear_layer
from tqdm import tqdm
nb = 1024
samples = 1
num_out=10
depth = int(math.ceil(math.log(nb*1.0, 2)))
inputs =enc_mat(name='nn_input'+str(idx)+'_', size= (1, nb))
outputs = enc_mat(name='nn_outputs'+str(idx)+'_', size=(num_out, 1))
weight = (np.random.uniform(size=(num_out, nb))> 0.5).astype(int)
layer = linear_layer(name='linear', weight=weight,
                    inputs=inputs, outputs=outputs)
layer_obj = circuit('linear_layer', layer, const_inputs=[])
layer_obj.write_file(filename='./test_linear.sheep')
processing_time = np.zeros(samples)
for idx in tqdm(range(0, samples)):
    inp_arr_val = list(map(lambda x : int(x), np.random.uniform(size=nb*1)> 0.5))
    inp_dict = inputs.get_input_dict(inp_arr_val)
    inputs_file = benchmark_utils.write_inputs_file(inp_dict)
    results = benchmark_utils.run_circuit('./test_linear.sheep',inputs_file,"bool","TFHE",eval_strategy='parallel')
    processing_time[idx] = results['Processing times (s)']['circuit_evaluation']
    out_vars = outputs.get_variables()
    xor_val = (weight == np.asarray([x for x in inp_arr_val]))
    #print(xor_val)
    out_bit_vec = np.asarray([int(results['Outputs'][var]) for var in out_vars])
    true_bit_vec = (np.sum(np.asarray(xor_val), axis=1)>nb/2).astype(int)
    assert np.alltrue(out_bit_vec == true_bit_vec)
print(processing_time.mean())

100%|██████████| 1/1 [08:13<00:00, 493.40s/it]

486.0


|Input Dim | Output Dim | Time|
|:---------| ---------- |:---:|
|100       |    10      | 48s |
|500       |    2       | 47s |
|1024      |    10      | 487s|